# Web Scraping

In [1]:
import requests
import sys
import re
from bs4 import BeautifulSoup as bs
import pandas as pd, numpy as np
import warnings
warnings.filterwarnings(action='once')
from icecream import ic
from random import randint, random
from time import sleep
from timeit import default_timer as timer
from playsound import playsound

In [2]:
url = "https://www.metacritic.com"
# first_page = "https://www.metacritic.com/browse/games/score/metascore/all/all/filtered?view=condensed"
user_agent = {'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:99.0) Gecko/20100101 Firefox/99.0'}
# review_dict = {'name':[], 'date':[], 'game':[], 'rating':[], 'review':[]}
# game_dict = {'game':[], 'release_date':[], 'genre':[], 'platforms':[], 'developer':[], 'esrb_rating':[], 'ESRBs':[], \
#              'metascore':[], 'userscore':[],'critic_reviews':[], 'user_reviews':[], 'num_players':[], 'summary':[] }

In [3]:
def get_details(soup, text, col, idx=1):
    details = []
    if soup.find('th', scope='row', text = text)==None:
        game_dict[col].append(None)
        return
    for i in soup.find('th', scope='row', text = text).next_siblings:
        details.append(i.text)
    details_ = ' '.join(details[idx].strip().split())
    game_dict[col].append(details_)

In [4]:
def get_platforms(soup):
    if soup.find('span', class_="label", string = "Also On:")==None:
        game_dict['platforms'].append(soup.find('span', class_="platform").text.strip())
    else:
        consoles = [soup.find('span', class_="platform").text.strip()]
        other_consoles = [console.text  for console in soup.find_all('a', class_="hover_none")[1:]]
        consoles.extend(other_consoles)
        game_dict['platforms'].append(', '.join(consoles))

In [5]:
def get_user_reviews(soup):
    if soup.find_all('span', class_ = 'count')[1].find('a')!=None:
        game_dict['user_reviews'].append(soup.find_all('span', class_ = 'count')[1].find('a').text[:-8])
    else:
        game_dict['user_reviews'].append(None)

In [6]:
def get_numplayers(soup):
    P = soup.find(text = 'Number of Players:')
    O = soup.find(text = 'Number of Online Players:')
    if P==None:
        if O==None or O.next_element.text=='No Online Multiplayer':
            game_dict['num_players'].append('1 Player')
            return
        else:
            game_dict['num_players'].append(O.next_element.text)
            return
    else:
        if O==None or O.next_element.text=='No Online Multiplayer':
            game_dict['num_players'].append(P.next_element.text)
            return
        else:
            players = [P.next_element.text, O.next_element.text]
            game_dict['num_players'].append(', '.join(players))
            return

In [7]:
def get_summary(soup):
    if soup.find('span', class_="label", string = "Also On:")!=None:
        if len(soup.find_all('span', class_ = 'data'))==4:
            game_dict['summary'].append(soup.find_all('span', class_ = 'data')[3].text)
        else:
            game_dict['summary'].append(None)
    else:
        if len(soup.find_all('span', class_ = 'data'))==3:
            game_dict['summary'].append(soup.find_all('span', class_ = 'data')[2].text)
        else:
            game_dict['summary'].append(None)

In [15]:
def parse_info(details_soup):
    # Get to Details Page
    
#     testing
#     if soup.find('a', attrs = {'class':'action'}, text = 'Details & Credits')==None:
#         print('SOUP')
#         print(soup)
#         return 'Done'
#     print(soup.find('a', attrs = {'class':'action'}, text = 'Details & Credits'))
#     return
    
#     details_link = url + soup.find('a', attrs = {'class':'action'}, text = 'Details & Credits').get('href')
#     details_response = requests.get(details_link, headers = user_agent)
#     details_soup = bs(details_response.text, 'html.parser')
    
    count = 1 
    while details_soup.a==None and count<3:
        print('Error, Trying DETAILS Again')
        sleep(3)
        details_response = requests.get(details_link, headers = user_agent)
        details_soup = bs(details_response.text, 'html.parser')
        count += 1
        print('Still None?:', details_soup.a==None)
        print('response code:', details_response.status_code)
    
#     Get game Details
    try:
        for detail in details_soup.find_all('div', id='main'):
            game_dict['title'].append(detail.find('h1').text)
            game_dict['release_date'].append(detail.find_all('span', class_='data')[1].text)
            get_details(detail, 'Genre(s):', 'genre')
    #         get_platforms(detail)
            game_dict['platforms'].append(detail.find('span', class_="platform").text.strip())
            get_details(detail, 'Developer:', 'developer')
            get_details(detail, 'Rating:', 'esrb_rating')
            get_details(detail, 'ESRB Descriptors:', 'ESRBs', idx=0)
            game_dict['metascore'].append(detail.find('span', itemprop='ratingValue').text)
            game_dict['userscore'].append(detail.find_all('a', class_='metascore_anchor')[1].contents[1].text)
            game_dict['critic_reviews'].append([i.find_all('span') for i in 
                                                detail.find_all('span', class_='count')][0][2].text.strip())
            get_user_reviews(detail)
            get_numplayers(detail)
            get_summary(detail)
    except Exception as e:
        print('\n',sys.exc_info()[0],'|||', sys.exc_info()[1], '||| Error on line', sys.exc_info()[2].tb_lineno)
        print('Info Failed')
        playsound('ding.mp3')
        return 'done'

In [34]:
def parse_reviews(user_review_soup):
    #  Extracts link to User Reviews Page from main game page
#     for user_review in soup.find_all('a', attrs = {'class':'action'}, text = 'User Reviews'):
#         user_review_link = url + user_review.get('href')

# #             Connects to User Review page
#         user_review_response = requests.get(user_review_link, headers = user_agent)
#         user_review_soup = bs(user_review_response.text, 'html.parser')
        
        
        
    count = 1
    while user_review_soup.a==None and count<3:
        print('Error, Trying REVIEWS Again')
        sleep(3)
        user_review_response = requests.get(user_review_link+'/robots.text', headers = user_agent)
        user_review_soup = bs(user_review_response.text, 'html.parser')
        count += 1
        print('Still None?:', user_review_soup.a==None)
        print('response code:', user_review_response.status_code)

#           credit: Adeline Ong
#           https://towardsdatascience.com/web-scraping-metacritic-reviews-using-beautifulsoup-63801bbe200e

#             Loop through User Review Page and Extract Review Data into a dictionary
    try:
        for review in user_review_soup.find_all('div', class_='review_content'):
            if review.find('div', class_='name') == None:
                break
    #                 get_consoles(user_review_soup) 
            review_dict['game'].append(user_review_soup.find('h1').text)
            review_dict['name'].append(review.find('div', class_='name').find(recursive = True).text)
            review_dict['date'].append(review.find('div', class_='date').text)
            review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
            if review.find('span', class_='blurb blurb_expanded'):
                review_dict['review'].append(review.find('span', class_='blurb blurb_expanded').text)
            elif review.find('div', class_='review_body').find('span')==None:
                review_dict['review'].append(None)
            else:
                review_dict['review'].append(review.find('div', class_='review_body').find('span').text)
    except Exception as e:
        print('\n',sys.exc_info()[0],'|||', sys.exc_info()[1], '||| Error on line', sys.exc_info()[2].tb_lineno)
        print('Reviews Failed')
        playsound('ding.mp3')
        return 'done'

In [98]:
def parse_games(startpage_link, next_page, end_page, game_num_start, game_num_end = 101):  
    next_page = next_page
    
#   request from current page
    try:
        response = requests.get(startpage_link, headers = user_agent)
        soup = bs(response.text, 'html.parser')
        current_page = int(soup.find('span', class_='page_num').text)
        print(f'parsing page: {current_page}/{end_page}')
    except Exception as e:
        print(sys.exc_info()[0],'|||', sys.exc_info()[1], '||| Error on line', sys.exc_info()[2].tb_lineno)
        print(f'Cannot Connect to page {next_page-1}')
        playsound('ding.mp3')
        return
    
#     Loop through current page and extract links to game pages
    game_num = game_num_start
    for game in soup.find_all('a', attrs = {'class':'title'})[game_num - 1:game_num_end]:
        game_link = url + game.get('href')
        
#         Connects to game page summary
        sleep(1)
        try:
            game_response = requests.get(game_link, headers = user_agent)
            game_soup = bs(game_response.text, 'html.parser')
        except Exception as e:
            print('Failed to connect to game page')
            print('Game response code:', game_response.status_code)
            print(sys.exc_info()[0],'|||', sys.exc_info()[1], '||| Error on line', sys.exc_info()[2].tb_lineno)
            print('Trying again...')
            sleep(10)
            game_response = requests.get(game_link, headers = user_agent)
            game_soup = bs(game_response.text, 'html.parser')
            
        count = 1
        while game_soup.a==None and count<3:
            print('Error, Trying GAME Again')
            sleep(10)
            game_response = requests.get(game_link, headers = user_agent)
            game_soup = bs(game_response.text, 'html.parser')
            count += 1
            print('Still None?:', game_soup.a==None)
            print('response code:', game_response.status_code)
        if game_soup.a==None:
            print('Trying GAME another time')
            sleep(10)
            game_response = requests.get(game_link+'/robots.text', headers = user_agent)
            game_soup = bs(details_response.text, 'html.parser')
            print('Still None?:', game_soup.a==None)
            print('response code:', game_response.status_code)
            if game_soup.find('a')==None:
                print(f'Game {game_num} GAME Failed')
                playsound('ding.mp3')
                return
        print('Making Game Connections...', end = ' ')
        sleep(3)
        
        
        #         Connect to Details Page
        try:
            details_link = url + game_soup.find('a', attrs = {'class':'action'}, text = 'Details & Credits').get('href')
            details_response = requests.get(details_link, headers = user_agent)
            details_soup = bs(details_response.text, 'html.parser')
        except Exception as e:
            sleep(10)
            print('\n',sys.exc_info()[0],'|||', sys.exc_info()[1], '||| Error on line', sys.exc_info()[2].tb_lineno)
            print('Failed to connect to Details page')
            print('Details response code:', details_response.status_code)
            print('Trying Details again')
            details_response = requests.get(details_link, headers = user_agent)
            details_soup = bs(details_response.text, 'html.parser')
        
        count = 1
        while details_soup.a==None and count<3:
            print('Error, Trying Details Again')
            sleep(10)
            details_response = requests.get(details_link, headers = user_agent)
            details_soup = bs(details_response.text, 'html.parser')
            count += 1
            print('Still None?:', details_soup.a==None)
            print('Details response code:', details_response.status_code)
        if details_soup.find('a')==None:
            print('Trying Details last time')
            sleep(10)
            details_response = requests.get(details_link+'/robots.text', headers = user_agent)
            details_soup = bs(details_response.text, 'html.parser')
            if details_soup.find('a')==None:
                print(f'Game {game_num} Details Failed')
                print('Details response code:', details_response.status_code)
                playsound('ding.mp3')
                return
            
            
        #             Connects to User Review page
        try:
            user_review_link = url + game_soup.find('a', attrs = {'class':'action'}, text = 'User Reviews').get('href')
            user_review_response = requests.get(user_review_link, headers = user_agent)
            user_review_soup = bs(user_review_response.text, 'html.parser')
        except Exception as e:
            sleep(10)
            print('\n', sys.exc_info()[0],'|||', sys.exc_info()[1], '||| Error on line', sys.exc_info()[2].tb_lineno)
            print('Failed to connect to Reviews page')
            print('Reviews response code:', user_review_response.status_code)
            print('Trying Reviews again')
            user_review_response = requests.get(user_review_link, headers = user_agent)
            user_review_soup = bs(user_review_response.text, 'html.parser')
            
            
        count = 1
        while user_review_soup.a==None and count<3:
            print('Error, Trying Reviews Again')
            sleep(10)
            user_review_response = requests.get(user_review_link, headers = user_agent)
            user_review_soup = bs(user_review_response.text, 'html.parser')
            count += 1
            print('Still None?:', user_review_soup.a==None)
            print('Reviews response code:', user_review_response.status_code)
        if user_review_soup.a==None:
            print('Trying Reviews last time')
            sleep(10)
            user_review_response = requests.get(user_review_link+'/robots.text', headers = user_agent)
            user_review_soup = bs(user_review_response.text, 'html.parser')
            if user_review_soup.a==None:
                print(f'Game {game_num} Reviews Failed')
                playsound('ding.mp3')
                return
        
        
#         Get Game info
        if parse_info(details_soup)=='done':
            return
        
        
#         Get Game Reviews
        if parse_reviews(user_review_soup)=='done':
            return
        
        print(f'Game {game_num} completed')
        game_num +=1
        
    if next_page == (end_page+1):
        find_next_page = soup.find('a', class_='page_num', string = f'{next_page}')
        global startpage
        startpage = url + find_next_page.get('href')
        print(f'Done, Next Start Page: {startpage}')
        print('Copy and Paste this link or use startpage variable')
        playsound('ding.mp3')
        return startpage_link
    if next_page>(end_page+1):
        print('Increase End page')
        playsound('ding.mp3')
        return
    if current_page>=next_page:
        print('Increase Next page')
        playsound('ding.mp3')
        return
    sleep(3)
    
#     Recurse through the following pages
    find_next_page = soup.find('a', class_='page_num', string = f'{next_page}')
    nextpage_link = url + find_next_page.get('href')
    next_page += 1
    parse_games(nextpage_link, next_page, end_page, game_num_start = 1, game_num_end = 101)

In [70]:
# startpage='https://www.metacritic.com/browse/games/score/metascore/all/all/filtered?view=condensed&page=30'

In [99]:
startpage

'https://www.metacritic.com/browse/games/score/metascore/all/all/filtered?view=condensed&page=35'

In [90]:
# start = timer()
# try:
#     parse_games(startpage, next_page = 32, end_page = 35, game_num_start = 1, game_num_end = 101)
# except Exception as e:
#     print('\n', sys.exc_info()[0],'|||', sys.exc_info()[1], '||| Error on line', sys.exc_info()[2].tb_lineno)
#     playsound('ding.mp3')
    
# end = timer()
# minutes = int((end-start)//60)
# seconds = int((end-start)-(minutes*60))
# print(f'Elapsed Time: {minutes} minutes {seconds} seconds')

parsing page: 31/35
Making Game Connections... Game 1 completed
Making Game Connections... Game 2 completed
Making Game Connections... Game 3 completed
Making Game Connections... Game 4 completed
Making Game Connections... Game 5 completed
Making Game Connections... Game 6 completed
Error, Trying GAME Again
Still None?: False
response code: 200
Making Game Connections... Game 7 completed
Making Game Connections... Game 8 completed
Making Game Connections... Game 9 completed
Making Game Connections... Game 10 completed
Making Game Connections... Game 11 completed
Making Game Connections... Game 12 completed
Making Game Connections... Game 13 completed
Making Game Connections... Game 14 completed
Making Game Connections... Game 15 completed
Making Game Connections... Game 16 completed
Making Game Connections... Game 17 completed
Making Game Connections... Error, Trying Details Again
Still None?: False
Details response code: 200
Game 18 completed
Making Game Connections... Game 19 complet

In [ ]:
# ['game', 'release_date', 'genre', 'platforms', 'developer', 'esrb_rating', 'ESRBs', 'metascore', 'userscore', 
#  'critic_reviews', 'user_reviews', 'num_players', 'summary'])

In [91]:
games_df = pd.DataFrame(game_dict).drop_duplicates()
games_df

,title,release_date,genre,platforms,developer,esrb_rating,ESRBs,metascore,userscore,critic_reviews,user_reviews,num_players,summary
0,The Legend of Zelda: Ocarina of Time,"Nov 23, 1998","Action Adventure, Fantasy",Nintendo 64,Nintendo,E,Violence,99,9.1,22,6982.0,1 Player,"As a young boy, Link is tricked by Ganondorf, ..."
1,Tony Hawk's Pro Skater 2,"Sep 20, 2000","Sports, Alternative, Skateboarding",PlayStation,Neversoft Entertainment,T,Mild Animated Violence Mild Language,98,7.4,19,786.0,1-2 Players,As most major publishers' development efforts ...
2,Grand Theft Auto IV,"Apr 29, 2008","Action Adventure, Modern, Modern, Open-World",PlayStation 3,Rockstar North,M,Blood Intense Violence Partial Nudity Strong L...,98,7.8,64,4529.0,1 Player,[Metacritic's 2008 PS3 Game of the Year; Also ...
3,SoulCalibur,"Sep 8, 1999","Action, Fighting, 3D",Dreamcast,Namco,T,Animated Violence Suggestive Themes,98,8.5,24,373.0,1-2 Players,"This is a tale of souls and swords, transcendi..."
4,Grand Theft Auto IV,"Apr 29, 2008","Action Adventure, Modern, Modern, Open-World",Xbox 360,Rockstar North,M,Blood Intense Violence Partial Nudity Strong L...,98,7.9,86,3594.0,1 Player,[Metacritic's 2008 Xbox 360 Game of the Year; ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3497,Closure,"Mar 27, 2012","Action, Platformer, Platformer, 2D, 2D",PlayStation 3,Eyebrow Interactive,E,Mild Fantasy Violence,81,7.0,11,33,1 Player,"In the dark and mysterious world of Closure, o..."
3498,Kingdom Hearts HD 2.5 ReMIX,"Dec 2, 2014","Role-Playing, Action RPG, Miscellaneous, Compi...",PlayStation 3,Square Enix,E10+,None,81,8.9,58,480,1 Player,KINGDOM HEARTS HD 2.5 ReMIX is an HD remastere...
3499,Guild Wars 2: End of Dragons,"Feb 28, 2022","Role-Playing, Massively Multiplayer",PC,ArenaNet,T,None,81,7.7,11,85,Massively Multiplayer,Guild Wars 2: End of Dragons is the third expa...
3500,Tom Clancy's Splinter Cell Chaos Theory,"Mar 31, 2005","Action Adventure, Modern",GameCube,Ubisoft Montreal,M,Blood Strong Language Violence,81,8.2,28,23,1-2 Players,"As Sam Fisher, Third Echelon's most skilled Sp..."


In [92]:
# games_df.to_csv('metacritic_games_four.csv')

In [23]:
# game_dict = pd.read_csv('metacritic_games_four.csv').drop(['Unnamed: 0'],axis=1).to_dict('list')

In [93]:
reviews_df= pd.DataFrame(review_dict).drop_duplicates()
reviews_df

,name,date,game,rating,review
0,ThomasR.,"Apr 3, 2003",Combat Mission: Barbarossa to Berlin,9,Needs a brush-up on the graphics.
1,JoaquinD.,"Aug 25, 2003",Combat Mission: Barbarossa to Berlin,10,Perfect strategic and tactical WWII wargame. ...
2,MichaelDorosh,"Aug 19, 2011",Combat Mission: Barbarossa to Berlin,10,"Still the gold standard for WEGO, squad-based,..."
3,dinin70,"May 3, 2017",Combat Mission: Barbarossa to Berlin,9,I don't think it deserves 10/10 for a simple r...
4,BlueFalcon,"Apr 9, 2005",NHL 2002,6,"I was hyped about this release at the time, b..."
...,...,...,...,...,...
113615,LanceLabrin,"Sep 4, 2019",Dragon Quest Builders,8,Muy buen juego en el que puedes pasar horas cr...
113616,BenjaRPG,"May 20, 2020",Dragon Quest Builders,8,Dragon Quest Builders es un juego de construcc...
113617,Mithrandirkun,"Apr 8, 2020",Dragon Quest Builders,8,"Un juego estilo Minecraft, adaptado al univers..."
113618,Anonime,"Jul 10, 2020",Dragon Quest Builders,8,"Great game.\r Visually it is very beautiful, p..."


In [94]:
# reviews_df.to_csv('metacritic_reviews_four.csv')

In [25]:
# review_dict = pd.read_csv('metacritic_reviews_four.csv').drop(['Unnamed: 0'],axis=1).to_dict('list')

In [95]:
reviews = reviews_df.sort_values('name')
reviews['ids'] = reviews.groupby('name').ngroup()
check_uniques = reviews[['ids','name','game','rating','review']]

check_uniques[['ids']].value_counts()[check_uniques[['ids']].value_counts()>2]

ids  
48144    293
6143     203
2681     175
41252    117
52890    116
        ... 
40555      3
4984       3
6037       3
34844      3
63915      3
Length: 7296, dtype: int64

In [96]:
for key in list(game_dict.keys())[:]:
    print(key,'length is', len(game_dict[key]), end = ", ")
#     newlist = game_dict[key][:-3]
#     print(f'New {key}','length is', len(newlist))
#     game_dict[key] = newlist

title length is 3502, release_date length is 3502, genre length is 3502, platforms length is 3502, developer length is 3502, esrb_rating length is 3502, ESRBs length is 3502, metascore length is 3502, userscore length is 3502, critic_reviews length is 3502, user_reviews length is 3502, num_players length is 3502, summary length is 3502, 

In [97]:
for key in list(review_dict.keys())[:]:
    print(key,'length is', len(review_dict[key]), end = ", ")
#     newlist = review_dict[key][:-1]
#     print(f'New {key}','length is', len(newlist))
#     review_dict[key] = newlist

name length is 113620, date length is 113620, game length is 113620, rating length is 113620, review length is 113620, 

In [369]:
# def get_reviews_per_game(startpage_link, next_page, end_page, game_num_start, game_num_end = 101): 
#     next_page = next_page
#     print(f'parsing page: {next_page-1}/{end_page}')
    
#     try:
#         response = requests.get(startpage_link, headers = user_agent)
#         soup = bs(response.text, 'html.parser')
#         current_page = int(soup.find('span', class_='page_num').text)
#     except:
#         playsound('ding.mp3')
    
# #     Loop through current page and extract links to game pages
#     game_num = game_num_start
#     for game in soup.find_all('a', attrs = {'class':'title'})[game_num - 1:game_num_end]:
#         game_link = url + game.get('href')
        
# #         Connects to game page summary
#         sleep(3)
#         try:
#             game_response = requests.get(game_link, headers = user_agent)
#             game_soup = bs(game_response.text, 'html.parser')
#         except:
#             playsound('ding.mp3')
        
# #         testing
        
        
#         count = 1
#         while game_soup.a==None and count<2:
#             print('Error, Trying GAME Again')
#             sleep(3)
#             game_response = requests.get(game_link, headers = user_agent)
#             game_soup = bs(game_response.text, 'html.parser')
#             count += 1
#             print('Still None?:', game_soup.a==None)
#             print('response code:', game_response.status_code)
        
#         if game_soup.a==None:
#             sleep(3)
#             game_response = requests.get(game_link+'/robots.text', headers = user_agent)
#             game_soup = bs(details_response.text, 'html.parser')
#             if game_soup.find('a')==None:
#                 print(f'Game {game_num} GAME Failed')
#                 playsound('ding.mp3')
#                 return
            
            
# #         get number of reviews
#         try:
#             if 
        
#             num_games['num_games'].append(int(game_soup.find_all('p', class_='see_all')[1].text.split()[2]))
#             num_games['title'].append(game_soup.find('h1').text)    
            
            
#         print(f'Game {game_num} completed')
#         game_num +=1
        
#     if next_page == (end_page+1):
#         find_next_page = soup.find('a', class_='page_num', string = f'{next_page}')
#         startpage_link = url + find_next_page.get('href')
#         print(f'Done, Next Start Page: {startpage_link}')
#         print('Copy and Paste the link to start_page or use startpage_link variable')
#         playsound('ding.mp3')
#         return startpage_link
#     if next_page>(end_page+1):
#         print('Increase End page')
#         playsound('ding.mp3')
#         return
#     if current_page>next_page:
#         print('Increase Next page')
#         playsound('ding.mp3')
#         return
#     sleep(3)
    
# #     Recurse through the following pages
#     find_next_page = soup.find('a', class_='page_num', string = f'{next_page}')
#     nextpage_link = url + find_next_page.get('href')
#     next_page += 1
#     get_reviews_per_game(nextpage_link, next_page, end_page, game_num_start = 1, game_num_end = 101)